In [1]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio



# mapping to the modules that make the app
sys.path.insert(0, "../..")
# sys.path.insert(0, "..")


In [2]:
from app.objects import structures

from notebooks.helpers import test_queries

In [3]:
ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

import app.models as models

c = models.CosmosdbClient()

executing local windows deployment


In [4]:
buildings = yaml.safe_load(
    open(os.path.join(os.getenv("abspath"), "app", "configurations", "buildings.yaml"))
)
buildings["explanation"]

[{'type': "all buildings have a 'type'/'label' of building"},
 {'name': "name_with_underscore. UI can remove the '_' at display time."},
 {'augments_resource': 'OPTIONAL augments a specific resource, creates it if it does not exist'},
 {'built_by': 'type of object that can build the building'},
 {'owned_by': 'type of object that can own/maintain this building, assuming a relationhsip to that object'},
 {'changes': 'stats changed by the owner'},
 {'faction_augments': 'if owned by a pop in the faction,  raises the stat for all pops in factions'},
 {'effort': 'time in `universal time units` that the action will take to complete'},
 {'requires_attr': 'required attributes of the agent. Determines if the agent can do the action.'},
 {'render_type': "guides the rendering in buildings.js. Available types: 'block'"}]

# Creating a new Building

In [5]:
username = "BillmanLocal2"
pop = test_queries.get_random_pop(c,username)
pop


{'isIdle': 'true',
 'name': 'Waquimac Lamdonzobat',
 'objid': '1992522165129',
 'conformity': 0.49,
 'literacy': 0.432,
 'aggression': 0.473,
 'constitution': 0.546,
 'health': 0.7,
 'isIn': '4921596451746',
 'industry': 0.5095,
 'wealth': 0.4708,
 'factionLoyalty': 0.375,
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '1992522165129'}

In [6]:
farm = buildings["buildings"][0]
farm["label"] = "building"
farm

{'name': 'farmland',
 'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
 'planet_requirements': {'isHabitable': 'true'},
 'augments_resource': {'vegitable_foodstuff': 10},
 'changes': {'wealth': -0.01},
 'owned_by': 'pop',
 'effort': 1,
 'render_type': 'block',
 'label': 'building'}

Should be able to search the building type by name

In [7]:
b = [b for b in buildings['buildings'] if b['name'] == "farmland"][0]
b

{'name': 'farmland',
 'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
 'planet_requirements': {'isHabitable': 'true'},
 'augments_resource': {'vegitable_foodstuff': 10},
 'changes': {'wealth': -0.01},
 'owned_by': 'pop',
 'effort': 1,
 'render_type': 'block',
 'label': 'building'}

In [8]:
building = structures.Building(pop,b)
building

<building: None; 8330456965074; farmland>

In [9]:
building.get_data()

{'name': 'farmland',
 'objid': '8330456965074',
 'label': 'building',
 'owns': '1992522165129',
 'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
 'planet_requirements': {'isHabitable': 'true'},
 'augments_resource': {'vegitable_foodstuff': 10},
 'changes': {'wealth': -0.01},
 'owned_by': 'pop',
 'effort': 1,
 'render_type': 'block'}

In [10]:
building.get_owned_by()

{'node1': '8330456965074', 'node2': '1992522165129', 'label': 'owns'}

In [11]:
data = {'nodes':[building.get_data()],'edges':[building.get_owned_by()]}
c.upload_data(userguid,data)

In [12]:
building_query = f"""
g.V().has('objid','{pop['objid']}').as('pop')
    .in('owns').as('building')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap('objid','name','changes','augments_resource','planet_requirements','description','render_type'))
"""

c.run_query(building_query)
c.res

[{'labels': [['pop'], ['building']],
  'objects': [{'objid': ['1992522165129'], 'name': ['Waquimac Lamdonzobat']},
   {'objid': ['8330456965074'],
    'name': ['farmland'],
    'changes': ['{wealth: -0.01}'],
    'augments_resource': ['{vegitable_foodstuff: 10}'],
    'planet_requirements': ['{isHabitable: true}'],
    'description': ['Generates organic foodstuffs, populations will consume food before consuming natural resources'],
    'render_type': ['block']}]}]